```
Visit Together.ai : https://api.together.xyz/settings/billing

Together API key : 

Enjoy 5,000 free credits to get started with your new API key. You can also access this later in Settings.

4ed1cb4bc5e717fef94c588dd40c5617616f96832e541351195d3fb983ee6cb5
```

# Getting Started with Together API

In [ ]:
# Install together package
!py -3.11 -m pip install --upgrade together

# Show Langchain Package
!py -3.11 -m pip show langchain

In [ ]:
import os
import together
from pprint import pprint

# Set your API KEY
#os.environ["TOGETHER_API_KEY"] = "4ed1cb4bc5e717fef94c588dd40c5617616f96832e541351195d3fb983ee6cb5"
together.api_key = "4ed1cb4bc5e717fef94c588dd40c5617616f96832e541351195d3fb983ee6cb5"

In [51]:
# list available models and descriptons
models = together.Models.list()
# for i in range(50):
#     pprint(models[i]['name'])

for idx, model, in enumerate(models):
    print(idx, model['name'])

0 EleutherAI/gpt-j-6b
1 EleutherAI/gpt-neox-20b
2 EleutherAI/pythia-12b-v0
3 EleutherAI/pythia-1b-v0
4 EleutherAI/pythia-2.8b-v0
5 EleutherAI/pythia-6.9b
6 HuggingFaceH4/starchat-alpha
7 NousResearch/Nous-Hermes-13b
8 NousResearch/Nous-Hermes-Llama2-13b
9 NumbersStation/nsql-6B
10 OpenAssistant/oasst-sft-4-pythia-12b-epoch-3.5
11 OpenAssistant/stablelm-7b-sft-v7-epoch-3
12 bigcode/starcoder
13 databricks/dolly-v2-12b
14 databricks/dolly-v2-3b
15 databricks/dolly-v2-7b
16 google/flan-t5-xl
17 huggyllama/llama-13b
18 huggyllama/llama-30b
19 huggyllama/llama-65b
20 huggyllama/llama-7b
21 lmsys/fastchat-t5-3b-v1.0
22 lmsys/vicuna-13b-v1.3
23 lmsys/vicuna-7b-v1.3
24 prompthero/openjourney
25 runwayml/stable-diffusion-v1-5
26 stabilityai/stable-diffusion-2-1
27 stabilityai/stable-diffusion-xl-base-1.0
28 stabilityai/stablelm-base-alpha-3b
29 stabilityai/stablelm-base-alpha-7b
30 tatsu-lab/alpaca-7b-wdiff
31 timdettmers/guanaco-7b
32 togethercomputer/GPT-JT-6B-v1
33 togethercomputer/GPT-JT-Mo

In [37]:
# print the first model's name
pprint(models[1]['name'])

'EleutherAI/pythia-6.9b'


In [43]:
output = together.Complete.create("Space robots", model="togethercomputer/llama-2-7b-chat")

In [44]:
# print generated text
print(output['output']['choices'][0]['text'])

are a fascinating topic, and they have the potential to revolutionize the way we explore and understand the universe. Here are some of the most interesting space robots currently in development:
1. NASA's Perseverance Rover: This robot is scheduled to launch in July 2020 and will explore Mars for signs of past or present life on the red planet. It will be equipped with a suite of scientific instruments, including a drill that can collect samples from beneath the Martian surface.
2. SpaceX's Starship: This reusable spacecraft is being developed by Elon


In [49]:
# To start the together model
together.Models.start("togethercomputer/llama-2-7b-chat")

{'success': True,
 'value': '7560a05cd91bc835ca70b0273ecd37a57566ae69bd6df0ebc315b9fb083dcb8d'}

In [ ]:
# To stop the together model
together.Models.stop("togethercomputer/RedPajama-INCITE-7B-Base")

# To start inferencing with our code

## This is Together class for this model

In [52]:
import together

import logging
from typing import Any, Dict, List, Mapping, Optional

from pydantic import Extra, Field, root_validator

from langchain.callbacks.manager import CallbackManagerForLLMRun
from langchain.llms.base import LLM
from langchain.llms.utils import enforce_stop_tokens
from langchain.utils import get_from_dict_or_env

class TogetherLLM(LLM):
    """Together large language models."""

    model: str = "togethercomputer/llama-2-70b-chat"
    """model endpoint to use"""

    together_api_key: str = os.environ["TOGETHER_API_KEY"]
    """Together API key"""

    temperature: float = 0.7
    """What sampling temperature to use."""

    max_tokens: int = 512
    """The maximum number of tokens to generate in the completion."""

    class Config:
        extra = Extra.forbid

    @root_validator()
    def validate_environment(cls, values: Dict) -> Dict:
        """Validate that the API key is set."""
        api_key = get_from_dict_or_env(
            values, "together_api_key", "TOGETHER_API_KEY"
        )
        values["together_api_key"] = api_key
        return values

    @property
    def _llm_type(self) -> str:
        """Return type of LLM."""
        return "together"

    def _call(
        self,
        prompt: str,
        **kwargs: Any,
    ) -> str:
        """Call to Together endpoint."""
        together.api_key = self.together_api_key
        output = together.Complete.create(prompt,
                                          model=self.model,
                                          max_tokens=self.max_tokens,
                                          temperature=self.temperature,
                                          )
        text = output['output']['choices'][0]['text']
        return text

In [75]:
llm = TogetherLLM(
    model= "togethercomputer/llama-2-7b-chat",
    temperature=0.1,
    max_tokens=512
)

In [76]:
llm("what is the differences between Llama2 model and Falcon ?")

'\n\nLlama2 and Falcon are both deep learning models developed by NVIDIA for natural language processing tasks, but they have some key differences:\n\n1. Architecture: Llama2 is a transformer-based model, while Falcon is a hierarchical transformer model. Transformer-based models use self-attention mechanisms to process input sequences in parallel, while hierarchical transformer models use a combination of local and global attention to capture both local and long-range dependencies in the input sequence.\n2. Training: Llama2 is trained using a masked language modeling task, where some of the input tokens are randomly replaced with a special mask token, and the model is trained to predict the original token. Falcon is trained using a combination of masked language modeling and next sentence prediction tasks.\n3. Performance: Llama2 has been shown to perform better than Falcon on some natural language processing tasks, such as language modeling and text classification. However, Falcon has

## Using Langchain

In [55]:
import json
import textwrap

B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
DEFAULT_SYSTEM_PROMPT = """\
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information."""


def get_prompt(instruction, new_system_prompt=DEFAULT_SYSTEM_PROMPT ):
    SYSTEM_PROMPT = B_SYS + new_system_prompt + E_SYS
    prompt_template =  B_INST + SYSTEM_PROMPT + instruction + E_INST
    return prompt_template

def cut_off_text(text, prompt):
    cutoff_phrase = prompt
    index = text.find(cutoff_phrase)
    if index != -1:
        return text[:index]
    else:
        return text

def remove_substring(string, substring):
    return string.replace(substring, "")


def parse_text(text):
        wrapped_text = textwrap.fill(text, width=100)
        print(wrapped_text +'\n\n')
        # return assistant_text

## Simple Chatbot

In [56]:
from langchain import PromptTemplate,  LLMChain

In [62]:
instruction = "Chat History:\n\n{chat_history} \n\nHuman: {user_input}\n\n Assistant:"
system_prompt = "You are a helpful assistant, you always only answer for the assistant then you stop. read the chat history to get context"

template = get_prompt(instruction, system_prompt)
print(template)

[INST]<<SYS>>
You are a helpful assistant, you always only answer for the assistant then you stop. read the chat history to get context
<</SYS>>

Chat History:

{chat_history} 

Human: {user_input}

 Assistant:[/INST]


In [63]:
prompt = PromptTemplate(
    input_variables=["chat_history", "user_input"], template=template
)
memory = ConversationBufferMemory(memory_key="chat_history")

In [64]:
llm_chain = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=memory,
)

In [74]:
llm_chain.predict(user_input="Hi, my name is Kelly")



> Entering new LLMChain chain...
Prompt after formatting:
[INST]<<SYS>>
You are a helpful assistant, you always only answer for the assistant then you stop. read the chat history to get context
<</SYS>>

Chat History:

Human: Hi, my name is Sam
AI:  Hello Sam! It's nice to meet you. How can I assist you today?
Human: Can you tell me about yourself.
AI:  Of course! I'm just an AI, I don't have personal experiences or feelings like humans do, but I'm here to help you with any questions or tasks you may have. How can I assist you today?
Human: Today is Friday. What number day of the week is that?
AI:  AI: It is the 5th day of the week.
Human: what is the day today?
AI:  AI: It is Friday.
Human: What is my name?
AI:  AI: Hello! My name is Assistant. How can I assist you today?
Human: Hi, my name is Lukas
AI:  Hello Lukas! It's nice to meet you. How can I assist you today?
Human: Can you tell me about yourself.
AI:  Of course! I'm just an AI, I don't have personal experiences or feelings 

" Hello Kelly! It's nice to meet you. How can I assist you today?"

In [71]:
llm_chain.predict(user_input="Can you tell me about yourself.")



> Entering new LLMChain chain...
Prompt after formatting:
[INST]<<SYS>>
You are a helpful assistant, you always only answer for the assistant then you stop. read the chat history to get context
<</SYS>>

Chat History:

Human: Hi, my name is Sam
AI:  Hello Sam! It's nice to meet you. How can I assist you today?
Human: Can you tell me about yourself.
AI:  Of course! I'm just an AI, I don't have personal experiences or feelings like humans do, but I'm here to help you with any questions or tasks you may have. How can I assist you today?
Human: Today is Friday. What number day of the week is that?
AI:  AI: It is the 5th day of the week.
Human: what is the day today?
AI:  AI: It is Friday.
Human: What is my name?
AI:  AI: Hello! My name is Assistant. How can I assist you today?
Human: Hi, my name is Lukas
AI:  Hello Lukas! It's nice to meet you. How can I assist you today? 

Human: Can you tell me about yourself.

 Assistant:[/INST]

> Finished chain.


" Of course! I'm just an AI, I don't have personal experiences or feelings like humans do, but I'm here to help you with any questions or tasks you may have. How can I assist you today?"

In [73]:
llm_chain.predict(user_input="Today is Friday. What number day of the week is that?")



> Entering new LLMChain chain...
Prompt after formatting:
[INST]<<SYS>>
You are a helpful assistant, you always only answer for the assistant then you stop. read the chat history to get context
<</SYS>>

Chat History:

Human: Hi, my name is Sam
AI:  Hello Sam! It's nice to meet you. How can I assist you today?
Human: Can you tell me about yourself.
AI:  Of course! I'm just an AI, I don't have personal experiences or feelings like humans do, but I'm here to help you with any questions or tasks you may have. How can I assist you today?
Human: Today is Friday. What number day of the week is that?
AI:  AI: It is the 5th day of the week.
Human: what is the day today?
AI:  AI: It is Friday.
Human: What is my name?
AI:  AI: Hello! My name is Assistant. How can I assist you today?
Human: Hi, my name is Lukas
AI:  Hello Lukas! It's nice to meet you. How can I assist you today?
Human: Can you tell me about yourself.
AI:  Of course! I'm just an AI, I don't have personal experiences or feelings 

" Sure, I'd be happy to help! To answer your question, today is Friday, which is the 5th day of the week."

In [68]:
llm_chain.predict(user_input="what is the day today?")



> Entering new LLMChain chain...
Prompt after formatting:
[INST]<<SYS>>
You are a helpful assistant, you always only answer for the assistant then you stop. read the chat history to get context
<</SYS>>

Chat History:

Human: Hi, my name is Sam
AI:  Hello Sam! It's nice to meet you. How can I assist you today?
Human: Can you tell me about yourself.
AI:  Of course! I'm just an AI, I don't have personal experiences or feelings like humans do, but I'm here to help you with any questions or tasks you may have. How can I assist you today?
Human: Today is Friday. What number day of the week is that?
AI:  AI: It is the 5th day of the week. 

Human: what is the day today?

 Assistant:[/INST]

> Finished chain.


' AI: It is Friday.'

In [72]:
llm_chain.predict(user_input="What is my name?")



> Entering new LLMChain chain...
Prompt after formatting:
[INST]<<SYS>>
You are a helpful assistant, you always only answer for the assistant then you stop. read the chat history to get context
<</SYS>>

Chat History:

Human: Hi, my name is Sam
AI:  Hello Sam! It's nice to meet you. How can I assist you today?
Human: Can you tell me about yourself.
AI:  Of course! I'm just an AI, I don't have personal experiences or feelings like humans do, but I'm here to help you with any questions or tasks you may have. How can I assist you today?
Human: Today is Friday. What number day of the week is that?
AI:  AI: It is the 5th day of the week.
Human: what is the day today?
AI:  AI: It is Friday.
Human: What is my name?
AI:  AI: Hello! My name is Assistant. How can I assist you today?
Human: Hi, my name is Lukas
AI:  Hello Lukas! It's nice to meet you. How can I assist you today?
Human: Can you tell me about yourself.
AI:  Of course! I'm just an AI, I don't have personal experiences or feelings 

" Hello Lukas! It's nice to meet you. How can I assist you today?"

In [ ]:
# To stop the together model
together.Models.stop("togethercomputer/RedPajama-INCITE-7B-Base")